In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
df=pd.read_csv('Reviews.csv')

In [4]:
#preparing dataset and cleaing

In [5]:
df_new=df[['Summary','Score']]

In [6]:
df_new.head()

,Summary,Score
0,Good Quality Dog Food,5
1,Not as Advertised,1
2,"""Delight"" says it all",4
3,Cough Medicine,2
4,Great taffy,5


In [7]:
df_new['Score'].replace({1: 'negative', 2: 'negative', 3: 'neutral', 4: 'positive', 5: 'positive'}, inplace=True)

In [8]:
df_new['Score'].value_counts()

positive    443777
negative     82037
neutral      42640
Name: Score, dtype: int64

In [9]:
df_new.isnull().sum()

Summary    27
Score       0
dtype: int64

In [10]:
df_new.dropna(subset=['Summary'], inplace=True)

In [11]:
df_new.isnull().sum()

Summary    0
Score      0
dtype: int64

In [12]:
df_new['Summary'] = df_new['Summary'].str.lower()

In [13]:
#spliting data into train,validation,test datasets
X_tr=df_new.loc[:0.8*(len(df_new)),'Summary']
Y_tr=df_new.loc[:0.8*(len(df_new)),'Score']
X_dev=df_new.loc[0.8*(len(df_new)):0.9*(len(df_new)),'Summary']
Y_dev=df_new.loc[0.8*(len(df_new)):0.9*(len(df_new)),'Score']
X_te=df_new.loc[0.9*(len(df_new)):,'Summary']
Y_te=df_new.loc[0.9*(len(df_new)):,'Score']

In [14]:
X_tr.shape,Y_tr.shape,  X_dev.shape,Y_dev.shape,   X_te.shape,Y_te.shape

((454719,), (454719,), (56840,), (56840,), (56868,), (56868,))

In [15]:
#rule based classification

In [16]:
#specifying rules
positive_words = ['large','delicious', 'amazing', 'excellent', 'fantastic', 'wonderful', 'tasty', 'awesome', 'great', 'love', 'perfect',
                  'enjoyable', 'yum', 'outstanding', 'superb', 'best', 'heavenly', 'satisfying', 'mouthwatering', 'terrific', 'top-notch','yummy','good','nice']
negative_words = ['small','not','disappointing', 'bad', 'poor', 'awful', 'horrible', 'mediocre', 'disgusting', 'terrible', 'unpleasant', 'bland',
                  'overrated', 'subpar', 'unappetizing', 'disliked', 'unimpressive', 'displeasing', 'underwhelming', 'rancid', 'worse', 'regrettable']
neutral_words = ['average', 'typical', 'ordinary', 'standard', 'common', 'usual', 'okay', 'acceptable', 'adequate', 'moderate',
                 'indifferent', 'so-so', 'plain', 'routine', 'inoffensive', 'fair', 'balanced', 'unspectacular', 'neutral', 'decent']


In [17]:
def rule_model(X,Y):
    #assigning scores
    k=0
    count=[0] * len(X)
    for i in X:
        words=i.split()
        for word in words:
            if word in positive_words:
                count[k]+= 10
            if word in negative_words:
                count[k]-=10
            if word in neutral_words:
                count[k]+=5
        k+=1
        
    #decision making    
    for i in range(len(count)):
        if count[i]>=10:
            count[i]='positive'
        elif count[i] >=0 and count[i] <=10:
            count[i]='neutral'
        else:
            count[i]='negative'
    
    #convert data frame to list
    ground_truth = Y.tolist()
    
    #accuracy for data
    pred=0
    for i in range(len(ground_truth)):
        if ground_truth[i]==count[i]:
            pred+=1
    accuracy=pred/len(X)
    
    return accuracy 

In [18]:
print('accuracy for train data is : {0} ' .format(rule_model(X_tr,Y_tr)))
print('accuracy for validation data is : {0} ' .format(rule_model(X_dev,Y_dev)))
print('accuracy for test data is : {0} ' .format(rule_model(X_te,Y_te)))

accuracy for train data is : 0.44286251509173796 
accuracy for validation data is : 0.44799437016185784 
accuracy for test data is : 0.44445030597172397 


In [19]:
#even if we randomly assign each review to positive,neural,negative we can get 33% accuracy ,here accuracy is around 44% 
# HOW TO INCREASE ACCURACY?
#make positive,negatuve,neutral lists effective and one can play with scores assignment.